In [43]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import re
from random import randint

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output

from tqdm import tqdm
from sklearn.utils import shuffle
import bz2
# from keras.layers import *
# from keras.models import Model
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Giving path of both datasets (training & testing ones)

AMAZON_REVIEW_DIR = 'C:/Users/Dell/DjangoProjects/datasets'
data = os.listdir(AMAZON_REVIEW_DIR)

print(data)

['test.ft.txt.bz2', 'train.ft.txt.bz2']


In [3]:
def reviewToY(review):
    return [1,0] if review.split(' ')[0] == '__label__1' else [0,1]

def reviewToX(review):
    review = review.split(' ', 1)[1][:-1].lower()
    review = re.sub('\d','0',review)
    if 'www.' in review or 'http:' in review or 'https:' in review or '.com' in review:
        review = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", review)
    return review

def splitReviewsLabels(lines):
    reviews = []
    labels = []
    for review in tqdm(lines):
        print('review=', review)
        review = reviewToX(review)
        label = reviewToY(review)
        reviews.append(review[:512])
        labels.append(label)
        print(reviews, labels)
    return reviews, labels

In [4]:
train_data = bz2.BZ2File(os.path.join(AMAZON_REVIEW_DIR, 'train.ft.txt.bz2'))
test_data = bz2.BZ2File(os.path.join(AMAZON_REVIEW_DIR,'test.ft.txt.bz2'))

In [5]:
train_lines = train_data.readlines()
test_lines = test_data.readlines()

In [6]:
train_lines = [x.decode('utf-8') for x in train_lines]
test_lines = [x.decode('utf-8') for x in test_lines]

In [7]:
del train_data, test_data
gc.collect()

304

In [17]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
    
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_lines]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_lines]

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])
                                                       
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [18]:
print(train_sentences[:5])

['stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^', "the best soundtrack ever to anything.: i'm reading a lot of reviews saying that this is the best 'game soundtrack' and i figured that i'd write a review to disagree a bit. this in my opinino is yasunori mitsuda's ultimate masterpiece. the music is timeless and i'm been listening to it for years now and its beauty simply refuses to fade.the price tag on this is pretty staggering i must say, but if you are going to buy any cd for this much money, this is the only one that i feel would be worth every penny.", 'amazing!: this soundtrack is my favorite music of all t

In [19]:
# del train_lines, test_lines
train_sentences[0]

'stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^'

In [20]:
# Create train and test dataframes

Na_train = {'Sentence': train_sentences, 'Label': train_labels}
Nav_train = pd.DataFrame(Na_train)

Na_test = {'Sentence': test_sentences, 'Label': test_labels}
Nav_test = pd.DataFrame(Na_test)

Nav_train.head()

Nav_train = Nav_train.head(900)
Nav_test = Nav_test.head(100)

In [21]:
print(Nav_train[:100])

                                             Sentence  Label
0   stuning even for the non-gamer: this sound tra...      1
1   the best soundtrack ever to anything.: i'm rea...      1
2   amazing!: this soundtrack is my favorite music...      1
3   excellent soundtrack: i truly like this soundt...      1
4   remember, pull your jaw off the floor after he...      1
..                                                ...    ...
95  very not worth your time: the book was wriiten...      0
96  very fun and educational: trains, shapes and p...      1
97  ludicrous and silly: i remember getting this b...      0
98  artistry: i think that the deodato concerts ar...      1
99  caution!: these tracks are not the "original" ...      0

[100 rows x 2 columns]


In [22]:
# SEPARATE NEGATIVE & POSITIVE REVIEWS 

test_pos = Nav_test[Nav_test['Label'] == 1]
test_pos = Nav_test['Sentence']
test_neg = Nav_test[Nav_test['Label'] == 0]
test_neg = Nav_test['Sentence']

In [23]:
del Na_train, Na_test, train_sentences, train_labels
gc.collect()

399

In [24]:
# Cleaning & extraction of features

sents = []
alll = []
stopwords_set = set(stopwords.words("english"))

for index, row in Nav_train.iterrows():
    words_filtered = [e.lower() for e in row.Sentence.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    sents.append((words_without_stopwords, row.Label))
    alll.extend(words_without_stopwords )

In [25]:
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features

In [26]:
w_features = get_word_features(alll)


In [27]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [28]:
# Training the Naive Bayes classifier
training_set = nltk.classify.apply_features(extract_features,sents)

In [29]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [30]:
train_pos = Nav_train[Nav_train['Label'] == 1]
train_pos = train_pos['Sentence']
train_neg = Nav_train[Nav_train['Label'] == 0]
train_neg = train_neg['Sentence']
test_pos = Nav_test[Nav_test['Label'] == 1]
test_pos = test_pos['Sentence']
test_neg = Nav_test[Nav_test['Label'] == 0]
test_neg = test_neg['Sentence']

In [31]:
test_neg.head(5)

2    batteries died within a year ...: i bought thi...
5    dvd player crapped out after one year: i also ...
6    incorrect disc: i love the style of this, but ...
7    dvd menu select problems: i cannot scroll thro...
9    not an "ultimate guide": firstly,i enjoyed the...
Name: Sentence, dtype: object

In [32]:
test_pos.head(5)

0    great cd: my lovely pat has one of the great v...
1    one of the best game music soundtracks - for a...
3    works fine, but maha energy is better: check o...
4    great for the non-audiophile: reviewed quite a...
8    unique weird orientalia from the 0000's: exoti...
Name: Sentence, dtype: object

In [33]:
neg_cnt = 0
pos_cnt = 0
for obj in test_neg: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 0): 
        neg_cnt = neg_cnt + 1
for obj in test_pos: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 1): 
        pos_cnt = pos_cnt + 1
        
print('[Negative]: %s/%s '  % (len(test_neg),neg_cnt))        
print('[Positive]: %s/%s '  % (len(test_pos),pos_cnt)) 

[Negative]: 47/42 
[Positive]: 53/40 


In [35]:
#aa = classifier.evaluate(Nav_test['Sentence'],Nav_test['Label'])
acccc= ((neg_cnt+pos_cnt)/(len(test_neg)+len(test_pos))) * 100
print("Accuracy by nltk classifier is", acccc)

Accuracy by nltk classifier is 82.0


In [36]:
print(test_neg.loc[52])
classifier.classify(extract_features(test_neg.loc[52].split()))

00 y/o potty humor from 00 somethings does not a comedy make....: saw this movie and felt compelled to warn people not to waste 000 agonizing minutes of their lives. in a nutshell, i have heard many more clever jokes and "one-liners" watching king of queens on cable.....seriously folks, it's that bad....the bad jokes and the 00 references to small jewish penises got old real quick. thought i would always be a sandler fan but he clearly, adam was in need of some rent money and jumped on this script for the cash and ran!why else would the plot line include seth rogan writing for him??? to put things into perspective, this film made bruno look like casablanca!!


0

In [42]:
obj = 'service was bad'
my_res = classifier.classify(extract_features(obj.split()))
print('Rating label is', my_res)

Rating label is 0


In [44]:
def getRatingLabel(review):

    label = classifier.classify(extract_features(review.split()))

    if label == 1:
        return randint(4, 5)
    elif label == 0:
        return randint(1, 3)

In [80]:
print(getRatingLabel('The painting work is very perfect'))

4
